![](https://raw.githubusercontent.com/sansty/my_data/master/predicted_cases.png)

Coronavirus disease (COVID-19) is an infectious disease caused by a new virus.
The disease causes respiratory illness (like the flu) with symptoms such as a cough, fever, and in more severe cases, difficulty breathing. You can protect yourself by washing your hands frequently, avoiding touching your face, and avoiding close contact (1 meter or 3 feet) with people who are unwell.
for more details please visit [WHO webpage](https://www.who.int/emergencies/diseases/novel-coronavirus-2019)

COCID-19 has seend more than 10000 active cases as on 14th April 2020 and it's spreading at various locations in India. The overall spread in increasing at the rate of cases getting doubled every fourth week.
This notebook aims to analyze the crowd source data from [covid19india](www.covid19india.org) and predict the cases for next week using different predictive functions.

On 14/04/20 : Predictions for India are :
#### 0	14 April	11311
#### 1	15 April	12336
#### 2	16 April	13397
#### 3	17 April	14493
#### 4	18 April	15619
#### 5	19 April	16774
#### 6	20 April	17954

### Working on other methods as well for prediction, will update daily with results.

### If you like, or have suggestions or found some errors in analysis please like or comment.. 
### working on prediction and comparison with COVID-19 spread in other countries and will be updating it soon..

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import plotly.express as px
from plotly.offline import init_notebook_mode, plot
from scipy.interpolate import make_interp_spline, BSpline
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import os

import warnings
warnings.filterwarnings('ignore')

import plotly.graph_objects as go
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import datetime
from datetime import date as datefun
from datetime import timedelta
import requests



# 1. Getting the date-set :
## Read data from [covid19india.org](http://covid19india.org) : A crowd source inititative to track the spread of virus in India 
The Data-set is fetched by the API provided at [api.covid19india.org](http://api.covid19india.org)

In [ ]:
data_raw=requests.get('https://api.covid19india.org/raw_data.json').json()  # read API
head=list(data_raw['raw_data'][0].keys())


### The API provides various information related to infected personlike detected city, district, gender etc.

In [ ]:
head # Keys

### 1.1 Getting the data-set to dataframe : 
### To convert the data to dataframe work done in this [github](https://github.com/anandsahadevan/COVID19india_visualize) repository is utilized. Author has created very good work to get the data in dataframe

In [ ]:
from shutil import copyfile

copyfile(src = "../input/my-data/util_functions.py", dst = "../working/util_functions.py")

from util_functions import *

In [ ]:

temp = pd.DataFrame([]) 
for i in range(0,len(data_raw['raw_data'])):
    data1=pd.DataFrame([data_raw['raw_data'][i].values()], columns=head)
    temp=temp.append(data1,ignore_index = True)

#------------  Remove No data rows ------------------------------------
temp1= list(temp.loc[0:len(data_raw['raw_data']),'currentstatus'])
valid_data=[i for i, item in enumerate(temp1) if item != '']
data_raw=temp[0:len(valid_data)]

Analyzing the raw data 

In [ ]:
data_raw 
for index, row in data_raw.iterrows():
    if row['detectedstate'] in 'Delhi':
            data_raw.at[index,'detecteddistrict'] = 'Delhi'


### Utiliity function as per [github](https://github.com/anandsahadevan/COVID19india_visualize) repository to convert the raw data in dataframes imported frome my_data directory

## Getting the state wise data and analyzing it by providing the range of dates

In [ ]:

start_date='2020-03-01'; end_date='2020-04-13'
df_state = get_state_df_from_api(data_raw,start_date,end_date)

#df_district = get_district_df_from_api(data_raw,start_date,end_date)
#df_district

1. Getting the dataset as per the latest date and current cases and Fatalities

In [ ]:
latest_date=df_state['Date'].max()
df_state_latest = df_state[df_state['Date'] == latest_date].sort_values(by='ConfirmedCases',ascending=False)
#df_state_= df_latest.groupby(['District']).sum().reset_index() # get sum of cases for each province
df_state_latest.style.background_gradient(cmap='Reds')

## Comparing the data-set from Data-set available in Kaggle

In [ ]:
df_k= pd.read_csv('../input/covid19-in-india/covid_19_india.csv')

df_k['Date']  = pd.to_datetime(df_k['Date'],format="%d/%m/%y") # new clean date columnn
df_k_latest = df_k[(df_k['Date'] == end_date)].sort_values(by='Confirmed',ascending=False)
#df_temp=df[df['Sno']==latest_index]
#latest_date=df_temp['Date'].max()
#df_latest = df[df['Date'] == latest_date]
df_k

## Observation : There is a slight differenc between the two cases which may be due to reporting of foreing nationals

In [ ]:
print("Total number of Cases According to Kaggle Data base as on ",end_date, "is = ",df_k_latest['Confirmed'].sum())
print("Total number of Cases According to Covid19India.org as on ",end_date, "is = ",df_state_latest['ConfirmedCases'].sum())

### For further analysis we will be using the data-set from covid19India.org

# Plotting the Trend of various states over time

### Plotting functions are taken from work done by [Traun Kumar](https://www.kaggle.com/tarunkr/covid-19-case-study-analysis-viz-comparisons) and modified for better visualization and more control

## Group and Arrange the data as state-wise 

In [ ]:
df_state=df_state.sort_values(by='Date',ascending=True)
#gb_state_time = df_state.groupby(['Date']).sum().reset_index()
gb_state_time = df_state.pivot_table(index=['Date'], 
            columns=['State'], values='ConfirmedCases').fillna(0)
gb_state_time['Date']=gb_state_time.index
gb_state_time.index.name = None
gb_state_time = gb_state_time.sort_values(by='Date',ascending=True)
gb_state_time['Date'] = pd.to_datetime(gb_state_time['Date'],format="%Y-%m-%d")
gb_state_time.index = pd.to_datetime(gb_state_time['Date'],format="%Y-%m-%d")
gb_state_time=gb_state_time.drop(['Date'],axis = 1) 
gb_state_time.tail(5)

In [ ]:
highlight=["Maharashtra","Tamil Nadu","Delhi","Telangana","Rajasthan"]
First_n=15
Days=30
Threshold=50
Label_X="Days ( Referenced to Threshold)"
Label_Y="Number of Confirmed Cases (Log Scale)"
Title="Trend Comparison of Different State (confirmed)\n Top 15 states in terms of COVID-19 cases"
plot_trend_rowdf(gb_state_time,Threshold,Days,First_n,highlight,Label_X,Label_Y,Title)

## Observations on India's trend
#### India is still in intital stage of infections and trends for each state shows varying pattern. India has initially shown very slow increase in rate of infection. but now it's trend is very close to increase in cases to twice every fourth day. 
#### Maharashtra State with most of the cases is also following the trend of cases get doubled every fourth day. Kerala state seems to stabilized. 
#### Tamilnadu and Delhi is showing very high increase in cases, and may surpass number of Maharashtra in terms of cases. Trends from Madhya Pradesh and Andhra Pradesh is also have higher trends as compared to other states.

### Trend for India goes to 30 Days while for states it's still in 7-10 days, which shows that new cases are getting identified in different states

### Prediction using curve fitting

In [ ]:
S = np.arange(0,200)
def predict(t,tau,theta,maxvalue):
    z = maxvalue*(1-np.exp(-(t-theta)/tau))
   # S[len(t)] = 0
    for i in range(len(t)):
        if t[i]<theta:
            S[i]=0
        else :
            S[i]=1
        z[i]=z[i]*S[i]
    return z

### Prediction at Country Level

In [ ]:
from scipy.optimize import curve_fit
def predict_n_plot_trend_for_Country(df,threshold,Days,Label_X,Label_Y,Title,df_next_week):
# modified from the awesome work by Tarun Kumar at https://www.kaggle.com/tarunkr/covid-19-case-study-analysis-viz-comparisons and modified for use
# Added the Prediction part
    
    temp_I = df#.sort_values(df.columns[-1], ascending= True)
    temp_I_sum = temp_I.sum(axis=1)
    last_row=temp_I.tail(1)
    last_row1 = last_row.sort_values(by=last_row.last_valid_index(),ascending=False, axis=1)
    f = plt.figure(figsize=(10,12))
    ax = f.add_subplot(111)
    x = Days
    t1_I = temp_I_sum.to_numpy()
    t2_I = t1_I[t1_I>threshold][:x]
    date = np.arange(1,len(t2_I[:x])+1)
    date1 = np.arange(1,len(t2_I[:x])+8)
    xnew = np.linspace(1, date.max(), Days)
    xnew1 = np.linspace(1, date1.max(), Days)
                
    a=int(len(t2_I))
    spl = make_interp_spline(date, t2_I, k=1)  # type: BSpline
    power_smooth1 = np.log2(t2_I)
    power_smooth1=power_smooth1*power_smooth1
    c,cov=curve_fit(predict,date[0:a],power_smooth1[0:a])
    power_smooth_predict=2**(np.sqrt(predict(date1,c[0],c[1],c[2])))
    df_next_week['India']=power_smooth_predict[len(t2_I):len(t2_I)+7]
               
    marker_style = dict(linewidth=4, linestyle='-', marker='o',markersize=6, markerfacecolor='#ffffff')
    plt.plot(date,t2_I,"-.",label = 'All The Cases',**marker_style)
    
    marker_style = dict(linewidth=4, linestyle='--', marker='o',markersize=6, markerfacecolor='#000000')
    plt.plot(date1,power_smooth_predict,"-.",label = 'Predicted Cases',**marker_style)
   
    plt.tick_params(labelsize = 14)        
    plt.xticks(np.arange(0,Days,7),[ "Day "+str(i) for i in range(Days)][::7])     

    # Reference lines 
    x = np.arange(0,Days/3)
    y = 2**(x+np.log(threshold))
    plt.plot(x,y,"--",linewidth =2,color = "gray")
    plt.annotate("No. of cases doubles every day",(x[-2],y[-1]),xycoords="data",fontsize=14,alpha = 0.5)

    x = np.arange(0,Days/2)
    y = 2**(x/2+np.log2(threshold))
    plt.plot(x,y,"--",linewidth =2,color = "gray")
    plt.annotate(".. every socend day",(x[-3],y[-1]),xycoords="data",fontsize=14,alpha = 0.5)

    x = np.arange(0,Days-4)
    y = 2**(x/7+np.log2(threshold))
    plt.plot(x,y,"--",linewidth =2,color = "gray")
    plt.annotate(".. every week",(x[-3],y[-1]),xycoords="data",fontsize=14,alpha = 0.5)

    x = np.arange(0,Days-4)
    y = 2**(x/30+np.log2(threshold))
    plt.plot(x,y,"--",linewidth =2,color = "gray")
    plt.annotate(".. every month",(x[-3],y[-1]),xycoords="data",fontsize=14,alpha = 0.5)

    x = np.arange(0,Days-4)
    y = 2**(x/4+np.log2(threshold))
    plt.plot(x,y,"--",linewidth =2,color = "Red")
    plt.annotate(".. every 4 days",(x[-3],y[-1]),color="Red",xycoords="data",fontsize=14,alpha = 0.8)

    # plot Params
    plt.xlabel(Label_X,fontsize=17)
    plt.ylabel(Label_Y,fontsize=17)
    plt.title(Title,fontsize=22)
    plt.legend(loc = "upper left")
    plt.yscale("log")
    plt.grid(which="both")
    plt.show()

In [ ]:
Days=30
Threshold=1000
Label_X="Days ( Referenced to Threshold)"
Label_Y="Number of Confirmed Cases (Log Scale)"
Title="Prediction of COVID-19 cases for Next Seven Days from 14/04/20"
df_next_week_1=pd.DataFrame()
df_next_week_1['Days']=['Today_Prediction','Day2_Prediction','Day3_Prediction','Day4_Prediction','Day5_Prediction','Day6_Prediction','Day7_Prediction']
predict_n_plot_trend_for_Country(gb_state_time,Threshold,Days,Label_X,Label_Y,Title,df_next_week_1)
df_next_week_1

## Prediction at State Level

In [ ]:
from scipy.optimize import curve_fit
def predict_n_plot_trend_rowdf(df,threshold,Days,First_n,highlight,Label_X,Label_Y,Title,df_next_week):
# modified from the awesome by Tarun Kumar work at https://www.kaggle.com/tarunkr/covid-19-case-study-analysis-viz-comparisons and modified for use
## Addded the prediction part
    
    temp_I = df#.sort_values(df.columns[-1], ascending= True)
    temp_I_sum = temp_I.sum(axis=1)
   # print(temp_I_sum)
    last_row=temp_I.tail(1)
    last_row1 = last_row.sort_values(by=last_row.last_valid_index(),ascending=False, axis=1)

 #   threshold = 50
 #   Days=51
    f = plt.figure(figsize=(10,12))
    ax = f.add_subplot(111)
    x = Days
    t1_I = temp_I_sum.to_numpy()
    t2_I = t1_I[t1_I>threshold][:x]
    date = np.arange(0,len(t2_I[:x]))
    xnew = np.linspace(date.min(), date.max(), Days)
    spl = make_interp_spline(date, t2_I, k=1)  # type: BSpline
    power_smooth = spl(xnew)
    marker_style = dict(linewidth=4, linestyle='-', marker='o',markersize=6, markerfacecolor='#ffffff')
    plt.plot(xnew,power_smooth,"-.",label = 'All The Cases',**marker_style)
          
    for i,col in enumerate(last_row1.columns):
        if col not in  ['Date','date']:
            x = Days
            threshold = int(temp_I[col].max()/10);
            if threshold <50:
                threshold=50
          #  print(threshold)
            t1_I = temp_I[col].to_numpy()
            t2_I = t1_I[t1_I>threshold][:x]
            
            if t2_I.size>3 :
                date = np.arange(1,len(t2_I[:x])+1)
                date1 = np.arange(1,len(t2_I[:x])+8)
                xnew = np.linspace(1, date.max(), Days)
                xnew1 = np.linspace(1, date1.max(), Days)
                
                a=int(len(t2_I))
                
                spl = make_interp_spline(date, t2_I, k=1)  # type: BSpline
                
                power_smooth = spl(xnew)
                power_smooth1 = np.log2(t2_I)
                power_smooth1 = power_smooth1*power_smooth1
                c,cov=curve_fit(predict,date[0:a],power_smooth1[0:a])
                power_smooth_predict=2**(np.sqrt(predict(date1,c[0],c[1],c[2])))
                df_next_week[col]=(power_smooth_predict[len(t2_I):len(t2_I)+7])
                
                if col in highlight:
                    marker_style = dict(linewidth=4, linestyle='-',markersize=6, markerfacecolor='#000000')
                    plt.plot(date,t2_I,"-.",label = col,**marker_style)
                    
                    marker_style = dict(linewidth=4, linestyle='--', marker='o',markersize=6, markerfacecolor='#000000')
                    plt.plot(date1,power_smooth_predict,"-.",label = col,**marker_style)
                

    plt.tick_params(labelsize = 14)        
    plt.xticks(np.arange(0,Days,7),[ "Day "+str(i) for i in range(Days)][::7])     

    # Reference lines 
    x = np.arange(0,Days/3)
    y = 2**(x+np.log(threshold))
    plt.plot(x,y,"--",linewidth =2,color = "gray")
    plt.annotate("No. of cases doubles every day",(x[-2],y[-1]),xycoords="data",fontsize=14,alpha = 0.5)

    x = np.arange(0,Days/2)
    y = 2**(x/2+np.log2(threshold))
    plt.plot(x,y,"--",linewidth =2,color = "gray")
    plt.annotate(".. every socend day",(x[-3],y[-1]),xycoords="data",fontsize=14,alpha = 0.5)

    x = np.arange(0,Days-4)
    y = 2**(x/7+np.log2(threshold))
    plt.plot(x,y,"--",linewidth =2,color = "gray")
    plt.annotate(".. every week",(x[-3],y[-1]),xycoords="data",fontsize=14,alpha = 0.5)

    x = np.arange(0,Days-4)
    y = 2**(x/30+np.log2(threshold))
    plt.plot(x,y,"--",linewidth =2,color = "gray")
    plt.annotate(".. every month",(x[-3],y[-1]),xycoords="data",fontsize=14,alpha = 0.5)

    x = np.arange(0,Days-4)
    y = 2**(x/4+np.log2(threshold))
    plt.plot(x,y,"--",linewidth =2,color = "Red")
    plt.annotate(".. every 4 days",(x[-3],y[-1]),color="Red",xycoords="data",fontsize=14,alpha = 0.8)

    # plot Params
    plt.xlabel(Label_X,fontsize=17)
    plt.ylabel(Label_Y,fontsize=17)
    plt.title(Title,fontsize=22)
    plt.legend(loc = "upper left")
    plt.yscale("log")
    plt.grid(which="both")
    plt.show()

In [ ]:
highlight=["Maharashtra","Tamil Nadu","Delhi","Telangana","Rajasthan"]
#highlight=["Maharashtra"]
First_n=15
Days=30
Threshold=75
Label_X="Days ( Referenced to Threshold)"
Label_Y="Number of Confirmed Cases (Log Scale)"
Title="TPrediction of Covid19 cases from 14/04/20 for different states"
df_next_week=pd.DataFrame()
df_next_week['Days']=['Today_Prediction','Day2_Prediction','Day3_Prediction','Day4_Prediction','Day5_Prediction','Day6_Prediction','Day7_Prediction']
predict_n_plot_trend_rowdf(gb_state_time,Threshold,Days,First_n,highlight,Label_X,Label_Y,Title,df_next_week)
df_next_week

In [ ]:
print("prediction for next seven days based on prediction for states\n",df_next_week.sum(axis=1))

# Working on more prediction will update soon with other models